In [1]:
import pandas as pd

In [2]:
def get_great_legelPerson(stockiid = 8422,legelperson = 1020 ):
    Price_Plus ,Price_Negative ,Price_Plus_number ,Price_Negative_number = [],[],[],[]
    stockiid , legelperson = str(stockiid),str(legelperson)
    CSV = './all_stock_price/public/{}.TW.csv'.format(stockiid)
    daily_trade =pd.read_csv('{}'.format(CSV))
    daily_trade.drop(['Dividends','Stock Splits'],axis = 1)
    daily_trade.index = pd.to_datetime(daily_trade['Date'],format = '%Y/%m/%d')
    try:
        tmp = pd.read_json(r'./{}/{}.json'.format(stockiid,legelperson))
        Locate = pd.read_json(r'./{}/{}.json'.format(stockiid,legelperson)).T.drop(0,axis = 0).drop(labels=[3,4],axis='columns')
        Locate['date_tmp'] = ''
        for i in range(1,len(Locate[0])+1):
            Locate['date_tmp'][i] = Locate[0][i].replace('/','-')
        Locate[1] = Locate[1].astype('float64')
        Locate[2] = Locate[2].astype('float64')
        Locate[0] = Locate['date_tmp']
        Locate = Locate.rename(columns={0:'Date' , 1: 'buy',2:'sell'})
        Locate = Locate.drop(['date_tmp'],axis = 1)
        Locate.index = pd.to_datetime(Locate['Date'],format = '%Y/%m/%d')    
        Locate['avg_price'] = (daily_trade['High']+daily_trade['Low'])*(0.5)
        Locate['blance'] = Locate['buy'] - Locate['sell']
        Locate['Cost'] = Locate['avg_price']*Locate['blance']

        for v in Locate['blance'].to_dict().values():
            if v > 0:
                Price_Plus_number.append(v)
            elif v < 0:
                Price_Negative_number.append(v)

        for p in Locate['Cost'].to_dict().values():
            if p > 0:
                Price_Plus.append(p)
            elif p < 0:

                Price_Negative.append(p)

        if len(Price_Plus) == 0:
            result_Price_Plus = 0
        else:
            result_Price_Plus = (sum(Price_Plus)/sum(Price_Plus_number))
        if len(Price_Negative) == 0:
            result_Price_Negative = 0
        else:
            result_Price_Negative = (sum(Price_Negative)/sum(Price_Negative_number))
        return [result_Price_Plus,result_Price_Negative]
    
    
    except ValueError as e:
        return [0.0,0.0]

In [5]:
City = pd.read_csv('./city_locate.csv').to_dict()
Index = {}

LegelPerson = pd.read_csv('./legelperson.csv')
for c in range(len(LegelPerson['Locate'])):
    LegelPerson['Locate'][c] = LegelPerson['Locate'][c][:3].replace('台','臺').replace('彰化市','彰化縣')\
             .replace('鳳山市','高雄市').replace('屏東市','屏東縣')\
             .replace('花蓮市','花蓮縣').replace('臺南縣','臺南市')\
             .replace('臺北縣','新北市').replace('桃園縣','桃園市')\
             .replace('臺中縣','臺中市').replace('高雄縣','高雄市')\
             .replace('宜蘭市','宜蘭縣').replace('苗栗市','苗栗縣')\
             .replace('臺東市','臺東縣').replace('臺東市','臺東縣') 
LegelPerson['Price_Plus'] = ''
LegelPerson['Price_Negative'] = ''
for c in range(len(LegelPerson['Locate'])):
    LegelPerson['Price_Plus'][c] = get_great_legelPerson(stockiid = 2330,legelperson = LegelPerson['id'][c] )[0]
    LegelPerson['Price_Negative'][c] = get_great_legelPerson(stockiid = 2330,legelperson = LegelPerson['id'][c] )[1]

# ---------------------------------------------------------------- #    
    
LegelPerson.to_csv('./Result/8422_to_legelperson_beta.csv')

In [31]:
LegelPerson = pd.read_csv('./Result/8422_to_legelperson_beta.csv').drop(['Unnamed: 0'],axis = 1)
LegelPerson['blance'] = LegelPerson['Price_Negative'] - LegelPerson['Price_Plus']

In [32]:
#挑選券商
Delete_List1 = []
# legel_name = pd.read_csv('./legel_name.csv').to_dict()
# DDic = {}
# for i in legel_name['legelpersoniid'].keys():
#     DDic[legel_name['name'][i]] = legel_name['legelpersoniid'][i]
# for key in DDic.keys():
#     if re.search("[-]", key) == None: 
#         Delete_List1.append(DDic[key])
        
        

# foreign_legelperson
# Delete_List2 = []
Delete_List2 = ['1360','1380','1440','1470','1480','1520','1530','1560','1570','1590','1650','8440','8890','8900','8960']

In [33]:
Delete_List3 = [i for i in LegelPerson[LegelPerson['Price_Plus'] == 0]['id']]  # 準備要刪除的rows(blance = 0)
Delete_List4 = [i for i in LegelPerson[LegelPerson['Price_Negative'] == 0]['id']]  # 準備要刪除的rows(blance = 0)

In [34]:
LegelPerson = LegelPerson.set_index(['id'])
LegelPerson['Count'] = 1 #設定計數
LegelPerson = LegelPerson.drop(Delete_List1 + Delete_List2 + Delete_List3 + Delete_List4)

In [35]:
LegelPerson

,Locate,Price_Plus,Price_Negative,blance,Count
id,,,,,
1020,臺北市,216.507661,188.991239,-27.516421,1
1021,臺中市,222.298469,216.779841,-5.518628,1
1022,臺南市,213.835267,202.688877,-11.146390,1
1023,高雄市,214.258950,206.088932,-8.170018,1
1024,嘉義市,219.693404,217.871111,-1.822293,1
1025,基隆市,236.565851,249.143487,12.577636,1
1028,彰化縣,228.224003,229.942516,1.718513,1
1029,高雄市,227.095875,212.579663,-14.516212,1
102A,新竹市,223.574634,216.777322,-6.797313,1


In [39]:
LegelPerson.sort_values(['blance'],ascending = (False))

,Locate,Price_Plus,Price_Negative,blance,Count
id,,,,,
9352,桃園市,154.202681,217.583437,63.380756,1
9297,臺中市,167.815293,213.318552,45.503258,1
779r,桃園市,200.605530,228.490058,27.884528,1
9186,新竹市,185.282655,203.019181,17.736526,1
116L,臺中市,212.563891,226.392508,13.828617,1
585B,新北市,210.284501,223.122487,12.837987,1
585b,臺北市,210.284501,223.122487,12.837987,1
1025,基隆市,236.565851,249.143487,12.577636,1
9369,新北市,227.047023,237.977612,10.930589,1


In [36]:
LegelPerson.groupby(['Locate',])['blance'].sum()

Locate
南投縣     -30.006504
嘉義市    -144.856832
嘉義縣     -28.977473
基隆市     -37.831786
宜蘭縣     -77.747284
屏東縣    -143.216926
彰化縣    -199.890012
新北市    -707.855026
新竹市    -162.706337
新竹縣     -68.396278
桃園市    -319.049383
澎湖縣     -11.082947
臺中市    -548.587044
臺北市   -1810.641497
臺南市    -430.502033
臺東縣      -4.467314
花蓮縣     -53.577777
苗栗縣     -85.718513
金門縣     -27.592906
雲林縣    -126.452283
高雄市    -715.363337
Name: blance, dtype: float64

In [37]:
LegelPerson.groupby(['Locate',])['Count'].sum()

Locate
南投縣     10
嘉義市     19
嘉義縣      6
基隆市     10
宜蘭縣     11
屏東縣     19
彰化縣     33
新北市    117
新竹市     26
新竹縣     15
桃園市     62
澎湖縣      2
臺中市     89
臺北市    222
臺南市     65
臺東縣      4
花蓮縣      7
苗栗縣     16
金門縣      2
雲林縣     19
高雄市     96
Name: Count, dtype: int64